In [2]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('chicago-crime-property.db') 

# Query all rows from the 'property_with_crime' table
query = 'SELECT * FROM property_with_crime'
property_with_crime_df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the property_with_crime DataFrame
print(property_with_crime_df)
property_with_crime_df.describe()


                style  beds  sqft  year_built  days_on_mls  sold_price  \
0       SINGLE_FAMILY     3  1350        1890          292     10155.0   
1       SINGLE_FAMILY     3  1300        1916           -1     57000.0   
2       SINGLE_FAMILY     3   692        1951          202     37000.0   
3        MULTI_FAMILY     7    -1          -1            2    175000.0   
4       SINGLE_FAMILY     2  1437        1922          122     21000.0   
...               ...   ...   ...         ...          ...         ...   
313414      TOWNHOMES     3   731        1962           -1     57000.0   
313415      TOWNHOMES     3   731        1967           67     55000.0   
313416  SINGLE_FAMILY     3  1318        1967           57    110000.0   
313417  SINGLE_FAMILY     4    -1        1969           52     90000.0   
313418  SINGLE_FAMILY     3  1152        1890           32     20000.0   

             last_sold_date   latitude  longitude       community  \
0       2014-02-11 00:00:00  41.737989 -87

,beds,sqft,year_built,days_on_mls,sold_price,latitude,longitude,crime_count
count,313419.000000,3.134190e+05,313419.000000,313419.000000,3.134190e+05,313419.000000,313419.000000,313419.000000
mean,2.156532,1.125788e+03,1304.297113,33.810659,7.236835e+05,41.881463,-87.674483,5147.207342
std,3.416618,6.579447e+03,916.872425,67.991046,5.971562e+06,0.084896,0.057103,3302.427075
min,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000e+00,41.644737,-87.935380,262.000000
25%,-1.000000,-1.000000e+00,-1.000000,-1.000000,1.890000e+05,41.824196,-87.706507,2368.000000
50%,2.000000,9.150000e+02,1915.000000,-1.000000,3.075000e+05,41.898391,-87.663889,4347.000000
75%,4.000000,1.600000e+03,1958.000000,53.000000,4.900000e+05,41.944243,-87.635302,7940.000000
max,999.000000,1.923400e+06,2023.000000,2144.000000,3.673000e+08,42.022841,-87.524883,12420.000000


In [3]:
import pandas as pd



# Calculate the total of the 'crime_count' column
total_crime_count = property_with_crime_df['crime_count'].sum()

print(f'Total Crime Count: {total_crime_count}')

Total Crime Count: 1613232578


In [3]:
print(property_with_crime_df.dtypes)

style              object
beds                int64
sqft                int64
year_built          int64
days_on_mls         int64
sold_price        float64
last_sold_date     object
latitude          float64
longitude         float64
community          object
neighborhood       object
crime_count         int64
dtype: object


In [8]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import sqlite3

# Connect to the database
conn = sqlite3.connect('chicago-crime-property.db')

# Read data from the database into a DataFrame
query = 'SELECT * FROM property_with_crime'
property_with_crime_df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

# Assuming X and y are your feature matrix and target variable
X = property_with_crime_df[['beds', 'sqft', 'year_built', 'days_on_mls', 'latitude', 'longitude', 'sold_price', 'community', 'neighborhood']]
y = property_with_crime_df['crime_count']

# Separate numerical and categorical features
numerical_features = ['beds', 'sqft', 'year_built', 'days_on_mls', 'latitude', 'longitude', 'sold_price']
categorical_features = ['community', 'neighborhood']

# Create transformers for numerical and categorical features
numerical_transformer = 'passthrough'
categorical_transformer = OneHotEncoder(drop='first')

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Apply preprocessing to the features
X_transformed = preprocessor.fit_transform(X)

# Calculate mutual information between features and target
mutual_info = mutual_info_regression(X_transformed, y)

# Create a DataFrame to store feature names and their mutual information scores
feature_info_df = pd.DataFrame({'Feature': preprocessor.get_feature_names_out(), 'Mutual_Information': mutual_info})

# Sort features by mutual information score
feature_info_df = feature_info_df.sort_values(by='Mutual_Information', ascending=False)

# Print the ranked features
print(feature_info_df)



                               Feature  Mutual_Information
4                        num__latitude            3.566893
5                       num__longitude            2.843895
2                      num__year_built            0.515788
6                      num__sold_price            0.354711
129  cat__neighborhood_NEAR NORTH SIDE            0.308795
..                                 ...                 ...
94          cat__neighborhood_BURNSIDE            0.009509
107      cat__neighborhood_FULLER PARK            0.008986
31          cat__community_FULLER PARK            0.008986
65            cat__community_RIVERDALE            0.005199
141        cat__neighborhood_RIVERDALE            0.005199

[159 rows x 2 columns]


In [1]:
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split, GridSearchCV  # Import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from scipy.stats.mstats import winsorize
import numpy as np

# Connect to the database
conn = sqlite3.connect('chicago-crime-property.db')

# Read data from the database into a DataFrame
query = 'SELECT * FROM property_with_crime'
property_with_crime_df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

# Replace -1 with NaN in the 'year_built' column
property_with_crime_df['year_built'].replace(-1, pd.NA, inplace=True)

# Convert the entire DataFrame to numeric
property_with_crime_df = property_with_crime_df.apply(pd.to_numeric, errors='ignore')

# Assuming X and y are your feature matrix and target variable
X = property_with_crime_df[['longitude', 'latitude', 'sold_price', 'year_built']]
y = property_with_crime_df['crime_count']

# Winsorize each column of X separately
X_winsorized = X.apply(lambda col: winsorize(col, limits=[0.01, 0.01]))

# Winsorize y
y_winsorized = winsorize(y, limits=[0.01, 0.01])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_winsorized, y_winsorized, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Impute missing values in the 'year_built' column
imputer = SimpleImputer(strategy='mean')
X_train_scaled[:, 3:4] = imputer.fit_transform(X_train_scaled[:, 3:4])
X_test_scaled[:, 3:4] = imputer.transform(X_test_scaled[:, 3:4])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'reg_alpha': [1e-5, 1e-4, 1e-3],
    'reg_lambda': [1e-5, 1e-4, 1e-3]
}

# Initialize XGBoost Regressor model with regularization
xgb_model = XGBRegressor(random_state=42)

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(xgb_model, param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Use the best parameters to train the final model
final_model = XGBRegressor(**best_params, random_state=42)
final_model.fit(X_train_scaled, y_train)

# Predict on the test data
y_pred_final = final_model.predict(X_test_scaled)

# Evaluate the final model performance
final_mse = mean_squared_error(y_test, y_pred_final)
print(f'Final Mean Squared Error: {final_mse}')

r2 = r2_score(y_test, y_pred_final)
print(f'R-squared Score: {r2}')

# Print the best parameters
print(f'Best Hyperparameters: {best_params}')

# Evaluate the model performance on the training set
train_mse = mean_squared_error(y_train, final_model.predict(X_train_scaled))
train_r2 = r2_score(y_train, final_model.predict(X_train_scaled))
print(f'Training Mean Squared Error: {train_mse}')
print(f'Training R-squared Score: {train_r2}')

# Evaluate the model performance on the test set
test_mse = mean_squared_error(y_test, y_pred_final)
test_r2 = r2_score(y_test, y_pred_final)
print(f'Test Mean Squared Error: {test_mse}')
print(f'Test R-squared Score: {test_r2}')

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Assuming y_test and y_pred_final are defined
mape = mean_absolute_percentage_error(y_test, y_pred_final)
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred_final))
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')


Final Mean Squared Error: 48623.753666707315
R-squared Score: 0.9955230129191749
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 150, 'reg_alpha': 1e-05, 'reg_lambda': 1e-05}
Training Mean Squared Error: 38377.8763329199
Training R-squared Score: 0.996479264967046
Test Mean Squared Error: 48623.753666707315
Test R-squared Score: 0.9955230129191749
Mean Absolute Percentage Error (MAPE): 3.40%
Root Mean Squared Error (RMSE): 220.51
